In [1]:
import chromadb
from chromadb.config import Settings
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
from sentence_transformers import SentenceTransformer

d:\Python3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chroma_db_path = '.\\chroma_db'

db = chromadb.PersistentClient(path=chroma_db_path, settings=Settings(allow_reset=True))
db_collection = db.get_or_create_collection(name='pdf_documents')

In [3]:
query = '推理小說'
ws_driver = CkipWordSegmenter(device=0)
pos_driver = CkipPosTagger(device=0)
sentenceModel = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def ckip_tokenize(text: str) -> str:
        def clean(sentence_ws, sentence_pos):
            # 這裡只保留名詞和動詞，且去除單一字元
            short_sentence = []
            stop_pos = set(['Nep', 'Nh', 'Nb'])
            for word_ws, word_pos in zip(sentence_ws, sentence_pos):
                is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
                is_not_stop_pos = word_pos not in stop_pos
                is_not_one_charactor = not (len(word_ws) == 1)
                if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
                    short_sentence.append(f"{word_ws}")
            return " ".join(short_sentence)
        
        global ws_driver, pos_driver
        ws = ws_driver([text])
        pos = pos_driver(ws)
        short = clean(ws[0], pos[0])
        return short

tokenizedQuery = ckip_tokenize(query)
queryVector = sentenceModel.encode(tokenizedQuery)

searchResult = db_collection.query(query_embeddings=[queryVector],
                                    n_results=2)
print(searchResult)

Inference: 100%|██████████| 1/1 [00:00<00:00, 71.52it/s]

{'ids': [['世界上最透明的故事（日本出版界話題作，只有紙本書可以體驗的感動）.pdf_5', '世界上最透明的故事（日本出版界話題作，只有紙本書可以體驗的感動）.pdf_7']], 'embeddings': None, 'documents': [[None, None]], 'uris': None, 'data': None, 'metadatas': [[{'file_name': '世界上最透明的故事（日本出版界話題作，只有紙本書可以體驗的感動）.pdf', 'text': '道 躋身於 推理 作家 行列 故事 推理 小說 紙本 詭計 世界 透明 故事 故事 編輯 譯者 費盡 心思 小說 中文版 面世 同樣 推理 評論 作家 冒業 身為 電子 書\n 愛好 割捨 紙本 以前 推給 紙本 氣味 觸感 這些 主觀 感受 現在 紙本 充滿 存在 書手 匠心 獨具 神乎其技 作品 知道 暴雷 期待 更多 一樣 接受 小說 帶來 震撼 感動 電子 推理 雜誌 主編 提醒 嚴禁 暴雷 世界 透明 故事 偵探 小說迷 福音 透明 紙本 書籍 代表 意義 緊張 情節 崩掉 思考 錯過 失去 顛覆 閱讀 極限 機會 高中 圖\n'}, {'file_name': '世界上最透明的故事（日本出版界話題作，只有紙本書可以體驗的感動）.pdf', 'text': ' 進軍 文藝 書市 神級 話題作 小說家 讀者 拋出 訊息 世界 透明 方式 溫柔 接住 譯者 泥蛙 聲稱 透明 看透 意外 失去 母親 認為 非親非故 血緣 關係 作家 留下 血腥 疑惑 迷霧 穿梭 心緒 扭曲 錯亂 實情 透明 細節 揭露 充滿 驚奇 紙本 呈現 不可 厲害 小\n 流轉 翻翻 流轉 一些 邪惡 褒義 書籍 需要 紙本 帶來 原汁 原味 體驗 身為 譯者 驚嘆 譯者 文字 運用 讀者 主角 抽絲剝繭 慢慢 拼湊 線索 謎底 揭曉 感動 餘韻 可謂 強烈 探究竟 書痴 下剋上 譯者 作者 編輯 譯\u3000 建構出 透明 故事'}]], 'distances': [[6.807389888299399, 7.053795909483816]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents